# Assignment  week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Nora Schinkel & Maarten Boon

__Student id(s)__ : 10416730 & 10764399

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='http://i.imgur.com/9g88in2.jpg%5C'/>
<img src='https://raw.githubusercontent.com/noracato/Kennissystemen/master/pledge.jpg'/>

# Infoboxes

* We willen voor elke Wikipedia pagina de informatie uit de infobox halen en die gestructureerd opslaan.
* In het simpele geval is de infobox een lijst van attribuut-waarde paren:
    * het zijn attributen van de "entiteit" waar de pagina over gaat.
    * Dit zijn dus typisch _RDF subject-predicate-object triples_
    * Die je in eerste orde logica als een binaire relatie zou opslaan (bijvoorbeeld _DateOfBirth(Albert-Speer,'19 maart 1905')_)
* Je zal zien dat hier flink wat haken en ogen aan zitten.
    



## Inspecteer een voorbeeld

* We gaan eens kijken hoe we de attribuut-waarde paren uit de zogenaamde _infoboxen_ uit de Wikipedia paginas kunnen gaan halen.
* Vergelijk eens het begin van de file `NLWikipedia/small.xml` met de pagina <https://nl.wikipedia.org/wiki/Albert_Speer>
* Die infobox informatie ziet er goed en redelijk eenvoudig te extraheren uit.
* Er zijn verschillende _mediawiki_ (het formaat van wikipedia paginas) parsers in omloop. 

### Je bent niet de eerste met deze opdracht ....

* Zoek op het web.
* Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
* Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers
* Denk ook eens aan <http://pandoc.org/demos.html>
* Zie ook [WikipediaInfoBoxes](WikipediaInfoBoxes) is de assignments folder.

## Vragen

## Download, en extraheer.  (33%)
* Download de dump met alle NL wikipedia paginas.


1. Verzin een goede datastructuur voor de subject-predicate-object triples die je uit de infoboxes per pagina gaat extraheren.
1. Schrijf een functie die uit het text element het infobox deel extraheert, en die lijst attribuut-waarde paren omzet naar jouw datastructuur.
2. Pas je functie toe op het [voorbeeld](NLWikipedia/sample.xml, en debug je code.

### Nou op de hele file/dump
1. Zoek uit hoe je een file stromend kunt inlezen in Python.
2. Pas nu je code toe op de hele file. 
3. Misschien  moet je wel  steeds gegevens naar een file gaan wegschrijven. 
4. Is jouw gekozen data formaat  handig?

#### Hints:
* Wil niet meteen alles tegelijk. Als er te veel variatie in de data zit, richt je dan op de meest voorkomende gevallen, en probeer die goed te krijgen. 
* Probeer wel een beeld te krijgen van "wat je mist", en beschrijf dat ook. 

## Analyseer  (33%)

1. Maak statistieken, wat zijn de meest gebruikte attributen, hoe vaak worden ze gebruikt, hoeveel attributen zijn er? Welke waardes worden er veel gebruikt? Hoeveel attributen zie je gemiddeld/mediaan per pagina? 
2. Doe een analyse op de waardes van de attributen? Kan je bepaalde types vinden?
3. Doe een analyse op de uniformiteit van de attribuutnamen. Uit de literatuur blijkt dat er erg veel verschillende manieren worden gebruikt om hetzelfde attribuut weer te geven (bijv, sterfdag, sterfdatum, sterftejaar, etc). Is dat ook zo in jouw dataset? Hoe kom je daar eigenlijk achter? 
 
 
## Reflecteer  (33%)
* Je hebt nu flink wat tijd besteed aan wat de mensen achter DBpedia, Yago, FreeBase en meer van dit soort initiatieven ook al doen. De bedoeling van de opdracht was om te ervaren wat een klus het is om een kennisbank te vullen met "gestructureerde" informatie van het web. 
* Beschrijf puntsgewijs wat volgens jullie de lastige onderdelen zijn van dit proces.
* Licht er 2-3 uit, en vertel hoe jullie het probleem hebben aangepakt.

# Mijn antwoorden

In [1]:
#dict van dicts, {"page":{"Subject":_sub_, "Predicate":_pred_, "Obj":_obj_}}???
import requests
import json
import re
from bs4 import BeautifulSoup


class RDF(object):
    def __init__(self, subject, predicate, obj):
        self.s = subject
        self.p = predicate
        self.o = obj

def get_name(infobox):
    name = infobox[1]
    name = re.split('=', name)[1]
    return name

url = 'http://maartenmarx.nl/teaching/KennisSystemen/Assignments/NLWikipedia/small.xml'
r = requests.get(url) 
content = r.content
content = content.decode('UTF-8')
soup = BeautifulSoup(content, 'xml')
text = soup.get_text()
better_text = re.split(r'\n', text)
better_text = re.split('{{', text)

all_triples = []
for paragraphs in better_text:
    if paragraphs.startswith('Info'):
        infobox = re.split('}}', paragraphs)[0]
        infobox = re.split(r'\n', infobox)
        subject = get_name(infobox)
        infobox.pop(0)
        infobox.pop(0)
        goodinfo = []
        for info in infobox:
            if not info.endswith('=') and not info.endswith('= ') and not info == '' and info.startswith('|'):
                twoparts = re.split('=', info)
                predicate = twoparts[0]
                predicate = re.split(' ', predicate)[1]
                obj = twoparts[1]
                triple = RDF(subject, predicate, obj)
                all_triples.append(triple)
                
   
        


In [ ]:
import matplotlib.pyplot as plt

#check amount of times attributes are used
def analyse (rdflist):
    all_attributes = []
    for rdf in rdflist:
        all_attributes.append(rdf.p)
    plot(all_attributes)
    
def plot(atlist):
    x = []
    y = []
    for atr in atlist:
        if atr in x:
            y[x.index(atr)] +=1
        else:
            x.append(atr)
            y.append(1)
            
    for times in y:
        if times == 1:
            index = y.index(times)
            y.pop(index)
            x.pop(index)
            
    plt.bar(range(len(y)), y, color='b', label='Attributes')
    plt.xlabel(x)
    plt.ylabel('Times used')
    plt.legend()
    plt.show()

analyse(all_triples)

In [5]:
#Get the average of "used" attributes per page
def average_attribute(rdflist):
    all_attributes = []
    for rdf in rdflist:
        all_attributes.append(rdf.p)
    total_attributes = len(all_attributes) #List of all attributes
    total_rdf = len(rdflist) #List od all
    avg = total_attributes/total_rdf
    return avg


print("average attributes per page", average_attribute(all_triples))

['afbeelding', 'onderschrift', 'gbdat', 'sterfdat', 'geslacht', 'gbplaats', 'sterfplaats', 'zijde', 'org', 'spfunc', 'ATP-id', 'afbeelding', 'bijnaam', 'afbeelding', 'partijvoorzitter', 'vice-partijvoorzitter', 'functienaam1', 'functie1', 'functienaam2', 'functie2', 'functienaam3', 'functie3', 'functienaam4', 'functie4', 'functienaam5', 'functie5', 'zetelnaam1', 'zetels1', 'zetelstotaal1', 'verkiezingen1', 'zetelnaam2', 'zetels2', 'zetelstotaal2', 'verkiezingen2', 'zetelnaam3', 'zetels3', 'zetelstotaal3', 'verkiezingen3', 'zetelnaam4', 'zetels4', 'zetelstotaal4', 'verkiezingen4', 'zetelnaam5', 'zetels5', 'zetelstotaal5', 'verkiezingen5', 'zetelnaam6', 'zetels6', 'zetelstotaal6', 'verkiezingen6', 'zetelnaam7', 'zetels7', 'zetelstotaal7', 'verkiezingen7', 'exnaam', 'afbeelding', 'onderschrift', 'volledigenaam', 'geboortedatum', 'geboorteplaats', 'sterfdatum', 'sterfplaats', 'nationaliteit', 'portaal', 'onderschrift', 'volledigenaam', 'geboortedatum', 'geboorteplaats', 'sterfdatum', 'ster

# Reflectie

Bij het parsen van de Infoboxen en het opzetten van de datastructuur is gebleken dat er veel ruis in de infoboxen zit, dit in de vorm van html tags, "lege" attributen enz. Dit zorgt voor een minder precieze analyse en ook voor dat de data onoverzichtelijk wordt voor mensen.
De opbouw van infoboxen is niet consistent (het gebruik van engelse en nederlandse termen door elkaar) zorgt voor een grote maat van onzuiverheid bij metingen
In het plot van hoe vaak de voorkomens zijn wordt bij veel dingen maar een voorkomen gemeten. Hierdoor vallen veel van de termen buiten het scherm. Dit wordt opgelost door alle termen die een keer voorkomen niet meer te laten zien. Dit is echter geen verantwoorde manier om de analyse toe te passen. 